In [ ]:
# ==========================================================
# Celda de Configuración Inicial para Google Colab
# ==========================================================
# Ejecuta esta celda al principio de tu notebook cada vez que inicies una nueva sesión.

# --- 1. Conectar a Google Drive ---
# Aparecerá una ventana para que autorices el acceso a tus archivos.
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Drive conectado exitosamente.")

# --- 2. Instalar Paquetes Necesarios ---
# Usamos -q para una instalación "silenciosa" (menos texto de salida).
# 'transformers' es la librería de Hugging Face para usar MathBERT.
# 'accelerate' ayuda a optimizar la ejecución en el hardware de Colab.
print("\n📦 Instalando librerías necesarias...")
!pip install -q transformers accelerate
print("✅ Librerías instaladas.")

Mounted at /content/drive

✅ Google Drive conectado exitosamente.

📦 Instalando librerías necesarias...
✅ Librerías instaladas.


In [ ]:
import os
import time
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# ==========================================================
# SECCIÓN DE CONFIGURACIÓN INTERACTIVA
# ==========================================================

# Opción 1: Menú Desplegable (RECOMENDADO)
# Es mejor porque evita errores al escribir el nombre del modelo.
model_selector = widgets.Dropdown(
    options=['mathbert', 'word2vec'],
    value='mathbert', # Valor por defecto
    description='Modelo:',
    disabled=False,
)

print("Por favor, selecciona el modelo a procesar y luego ejecuta las celdas de abajo.")
display(model_selector)

Por favor, selecciona el modelo a procesar y luego ejecuta las celdas de abajo.


Dropdown(description='Modelo:', options=('mathbert', 'word2vec'), value='mathbert')

In [ ]:
# ==========================================================
# 1. CONFIGURACIÓN
# ==========================================================
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/TFM_Cienciometria'
MODEL_NAME = model_selector.value # <-- ¡Aquí está la magia!

print(f"✅ Modelo seleccionado: {MODEL_NAME}")

# Configurar rutas y directorios basados en la selección
if MODEL_NAME == 'mathbert':
    TMP_DIR = os.path.join(DRIVE_PROJECT_PATH, 'data/mathbert/tmp')
elif MODEL_NAME == 'word2vec':
    TMP_DIR = os.path.join(DRIVE_PROJECT_PATH, 'data/word2vec/tmp') # Asegúrate que esta ruta exista
else:
    raise ValueError("Modelo no reconocido. Por favor, selecciona 'mathbert' o 'word2vec'.")

# El resto de tus parámetros pueden seguir aquí
BASE = "TODO"
MESES = 120
VENTANA_K = 150000
ESTRICTO = True
POOL_MIN = 50000
PERCENTILES = [0, 5, 10, 20]

# --- PARÁMETRO DE OPTIMIZACIÓN ---
# Con el enfoque híbrido, puedes usar un BATCH_SIZE grande sin temor a errores de memoria.
# 8192 o 16384 son buenos valores para una A100.
BATCH_SIZE = 16384

# ==========================================================
# 2. LÓGICA DE CÁLCULO HÍBRIDA (SOLUCIÓN DEFINITIVA)
# ==========================================================
def _get_suffix(meses, k, estricto, pool_min, percentiles):
    """Genera el sufijo para los nombres de archivo basado en la configuración."""
    parts = [f"p{'_'.join(str(p) for p in percentiles)}"]
    if k is not None: parts.append(f"k{k}")
    if meses is not None: parts.append(f"m{meses}")
    if estricto: parts.append("strict")
    if pool_min is not None: parts.append(f"pool{pool_min}")
    return "_".join(parts)

def compute_novelty_hybrid_A100(df: pd.DataFrame, percentiles, meses=None, k=None, estricto=False, pool_min=None, batch_size=8192):
    """
    Calcula la novedad usando un enfoque híbrido: procesa en lotes grandes para minimizar
    la sobrecarga de Python, pero calcula la novedad de cada artículo individualmente
    dentro del lote para evitar la creación de tensores intermedios masivos y prevenir
    errores de OutOfMemory.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🚀 Realizando cálculos HÍBRIDOS en {device} con precisión FP16 y BATCH_SIZE={batch_size}")

    df["anio_mes"] = pd.to_datetime(df["cover_date"], errors="coerce").dt.to_period("M")
    period_nums = df["anio_mes"].apply(lambda p: p.year * 12 + p.month if pd.notna(p) else -1).to_numpy()

    # Carga y normalización de vectores en la GPU
    V = np.vstack(df["vector"].values).astype(np.float32)
    V_gpu = torch.from_numpy(V).to(device, dtype=torch.float16)
    V_gpu = V_gpu / torch.norm(V_gpu, dim=1, keepdim=True).clamp(min=1e-12)

    usable = df["vector"].notna().to_numpy()
    q_tensor = torch.tensor([p / 100.0 for p in percentiles], device=device, dtype=torch.float32)

    results = []

    desc = f"🔍 Novedad HÍBRIDA-GPU (K={k or 'ALL'}, M={meses or 'ALL'})"
    # Bucle principal por lotes
    for i in tqdm(range(0, len(df), batch_size), desc=desc):
        batch_indices = np.arange(i, min(i + batch_size, len(df)))

        # Bucle secundario dentro del lote: aquí se evitan los errores de memoria
        for j in batch_indices:
            if not usable[j]:
                results.append({"eid": df.at[j, "eid"], "pool_size": 0, **{f"novelty_p{p}": np.nan for p in percentiles}})
                continue

            # Cálculo de la ventana de comparación (pool)
            current = period_nums[j]
            start = np.searchsorted(period_nums[:j], current - meses, side="left") if meses else 0
            end = np.searchsorted(period_nums[:j], current, side="left") if estricto else j
            valid_indices = np.where(usable[start:end])[0] + start

            if estricto and len(valid_indices) < (pool_min or 0):
                end = j
                valid_indices = np.where(usable[start:end])[0] + start

            if k is not None and len(valid_indices) > k:
                valid_indices = valid_indices[-k:]

            pool_size = len(valid_indices)

            # Cálculo de la novedad en la GPU
            if pool_size == 0:
                results.append({"eid": df.at[j, "eid"], "pool_size": 0, **{f"novelty_p{p}": np.nan for p in percentiles}})
            else:
                target_vector = V_gpu[j]
                pool_vectors = V_gpu[valid_indices]

                sims = pool_vectors @ target_vector
                dists = 1.0 - sims

                novelty_values = torch.quantile(dists.float(), q=q_tensor, interpolation='linear')

                novelty = {f"novelty_p{p}": novelty_values[idx].item() for idx, p in enumerate(percentiles)}
                results.append({"eid": df.at[j, "eid"], "pool_size": pool_size, **novelty})

    return pd.DataFrame(results)

# ==========================================================
# 4. EJECUCIÓN PRINCIPAL
# ==========================================================
if __name__ == '__main__':
    t0 = time.time()
    suffix = _get_suffix(MESES, VENTANA_K, ESTRICTO, POOL_MIN, PERCENTILES)
    input_path = os.path.join(TMP_DIR, f"04_vectores_{MODEL_NAME}_{BASE}.pkl")
    csv_output = os.path.join(TMP_DIR, f"05_novelty_scores_{MODEL_NAME}_{BASE}_{suffix}.csv")
    pkl_output = os.path.join(TMP_DIR, f"05_novelty_scores_{MODEL_NAME}_{BASE}_{suffix}.pkl")

    print(f"📦 Cargando vectores desde: {input_path}")
    df_main = pd.read_pickle(input_path)

    df_main = df_main.sort_values(["cover_date", "eid"]).reset_index(drop=True)

    print("🧠 Calculando índices de novedad (Optimizado Híbrido para A100)...")
    # --- LLAMADA A LA FUNCIÓN HÍBRIDA Y ROBUSTA ---
    novelty_df = compute_novelty_hybrid_A100(
        df_main, PERCENTILES,
        meses=MESES, k=VENTANA_K,
        estricto=ESTRICTO, pool_min=POOL_MIN,
        batch_size=BATCH_SIZE
    )

    print("\n📊 Resumen de novedad por percentil:")
    for p in PERCENTILES:
        col = f"novelty_p{p}"
        if col in novelty_df.columns:
            s = novelty_df[col].describe()
            print(f"    p{p}: count={int(s['count'])} mean={s['mean']:.4f} std={s['std']:.4f} min={s['min']:.4f} max={s['max']:.4f}")

    print("\n💾 Guardando resultados...")
    meta_cols = [c for c in ["eid", "title", "cover_date", "citedby_count", "doi", "text"] if c in df_main.columns]
    final_df = df_main[meta_cols].merge(novelty_df, on="eid", how="left")

    final_df.to_csv(csv_output, index=False)
    final_df.to_pickle(pkl_output)

    elapsed = time.time() - t0
    print(f"\n✅ Resultados guardados en:\n- {csv_output}\n- {pkl_output}")
    print(f"⏱️  Tiempo total: {elapsed/60:.2f} min")

✅ Modelo seleccionado: word2vec
📦 Cargando vectores desde: /content/drive/MyDrive/TFM_Cienciometria/data/word2vec/tmp/04_vectores_word2vec_TODO.pkl
🧠 Calculando índices de novedad (Optimizado Híbrido para A100)...
🚀 Realizando cálculos HÍBRIDOS en cuda con precisión FP16 y BATCH_SIZE=16384


🔍 Novedad HÍBRIDA-GPU (K=150000, M=120):   0%|          | 0/75 [00:00<?, ?it/s]


📊 Resumen de novedad por percentil:
    p0: count=1213939 mean=0.0647 std=0.0238 min=-0.0010 max=0.5654
    p5: count=1213939 mean=0.2025 std=0.0474 min=0.0648 max=0.5669
    p10: count=1213939 mean=0.2293 std=0.0502 min=0.0919 max=0.5903
    p20: count=1213939 mean=0.2645 std=0.0532 min=0.1118 max=0.6211

💾 Guardando resultados...

✅ Resultados guardados en:
- /content/drive/MyDrive/TFM_Cienciometria/data/word2vec/tmp/05_novelty_scores_word2vec_TODO_p0_5_10_20_k150000_m120_strict_pool50000.csv
- /content/drive/MyDrive/TFM_Cienciometria/data/word2vec/tmp/05_novelty_scores_word2vec_TODO_p0_5_10_20_k150000_m120_strict_pool50000.pkl
⏱️  Tiempo total: 31.99 min
